# Задача 1 Удвоение чисел и получение первого результата

In [1]:
# Получение данных из файла
import pathlib
data_path = pathlib.Path.cwd().joinpath('data/test_list_numbers.txt')
# Считаем содержимое файла в одну строку избавившись от переводов строк и пробелов
data_str = ""
with open(data_path, 'r') as f:
    for line in f.readlines():
        data_str += line.replace('\n','').replace(' ', '')
# убрать внешние скобки - останется только "содержимое" внешнего списка
data_str = data_str[1:-1] 
# разделить на "вложеные" списки
data_list = data_str.replace("[","").split("],")

def str_to_int_list(str_with_numbers):
    "Строку с числами преобразовать список чисел"
    l1 = str_with_numbers.split(",")
    s2i = lambda s: int(s) if s.isdigit() else None
    return list(map(s2i, l1))
# преобразовать в списки с числами
data_int = list(map(str_to_int_list,data_list))

In [2]:
import time
class ProcessNumberList:
    """Организация параллельной обработки списков чисел"""
    # флаг, того надо или нет прервать процесс обработки списков, выполняемых обработчиками конкретного экземпляра класса
    __need_stop_process_list = False

    def __init__(self):
        self.reset_need_stop_process_list()
        
    @property
    def need_stop_process_list(self):
        """Флаг необходимости прерывания соседних потоков, обрабатываемых в этом же объекте"""
        return self.__need_stop_process_list

    def reset_need_stop_process_list(self):
        """Сбросить флаг необходимости прерывания соседних потоков"""
        self.__need_stop_process_list = False

    def __process_number(self, number):
        """Обработка одно числа из списка"""
        result = number * 2
        time.sleep(0.1) # задержка выше, чем в задании, чтобы нагляднее была разница в длительности двух подходов
        return result

    def __process_list(self, numbers, break_after_first = False):
        """Обработка списка чисел"""
        result = 0
        for number in numbers:
            if break_after_first and self.__need_stop_process_list:
                # соседний поток обработал свой список - значит завершить и обработку текущего списка
                return None
            if number is not None:
                result += self.__process_number(number)
        if break_after_first:
            # текущий список завершили обрабатывать - сообщить об этом соседним потокам
            self.__need_stop_process_list = True
        return result

    def process_list_break_after_first(self, numbers):
        """Обработать список чисел, остановившись как только какой-нибудь поток обработает свой список"""
        return self.__process_list(numbers, break_after_first = True)

    def process_list_wait_all(self, numbers):
        """Обработать список чисел не обращая внимания на соседние потоки"""
        return self.__process_list(numbers, break_after_first = False)    


In [3]:
%%time
# Вариант без принудительного останова параллельных потоков
from concurrent.futures import ThreadPoolExecutor, as_completed
first_list_sum = None
# Запуск без принудительного останова параллельных потоков
with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    p = ProcessNumberList()
    futures = [executor.submit(p.process_list_wait_all, l) for l in data_int]

    # Получение результатов задач
    for future in as_completed(futures):
        first_list_sum = future.result()
        break # больше результатов можно не ждать - прерываем цикл 
print(f"Сумма чисел в первом обработанном списке: {first_list_sum}")

Сумма чисел в первом обработанном списке: 11090
CPU times: user 5.61 ms, sys: 2.23 ms, total: 7.84 ms
Wall time: 2.8 s


In [4]:
%%time
# Вариант с принудительным остановом обработки в параллельных потоках
from concurrent.futures import ThreadPoolExecutor, as_completed
first_list_sum = None
with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    p = ProcessNumberList()
    futures = [executor.submit(p.process_list_break_after_first, l) for l in data_int]

    # Получение результатов задач
    for future in as_completed(futures):
        first_list_sum = future.result()
        break # больше результатов можно не ждать - прерываем цикл 
print(f"Сумма чисел в первом обработанном списке: {first_list_sum}")

Сумма чисел в первом обработанном списке: 11090
CPU times: user 2.82 ms, sys: 4.98 ms, total: 7.8 ms
Wall time: 1.2 s


# Задача 2 Поиск и суммирование чисел через цепочку файлов

In [5]:
import pathlib
import zipfile
from multiprocessing import Pool
import platform

zip_with_filenames_path = pathlib.Path.cwd().joinpath('data/path_8_8.zip')
data_zip_path = pathlib.Path.cwd().joinpath('data/recursive_challenge_8_8.zip')

In [6]:
def prepared_filename(filename):
    """Преобразование стиль пути к файлу Linux <-> Windows"""
    if platform.system()=='Linux':
        filename = filename.replace('\\','/')    
    else:
        filename = filename.replace('/', '\\')    
    return filename

def process_file(filename, zip_with_filenames_path, data_zip_path):
    """Обработать один файл"""
    #TODO: по хорошему надо бы передавать не имена архивов, а уже объекты ZipFile. 
    #      Но так нагляднее разница в многопоточном и однопоточном подходе.
    #      И есть риски попасть на блокировки, если несколько потоков будут работать с одним объектом ZipFile - требуется больше изучения
    
    # считаем ссылку на файл с данными из нужного файла в архиве с файлами-ссылками
    with zipfile.ZipFile(zip_with_filenames_path) as zip_with_filenames:
        data_file_path = prepared_filename(zip_with_filenames.read(filename).decode())
    # найти файл в архиве с данными и считать из него число
    with zipfile.ZipFile(data_zip_path) as data_zip:
        num = int(data_zip.read(data_file_path).decode())
    return num


In [7]:
# Получение списка файлов
files = []
with zipfile.ZipFile(zip_with_filenames_path) as paths_file:
    files = [(p.filename, zip_with_filenames_path, data_zip_path) for p in paths_file.infolist() if not p.is_dir()]

In [8]:
%%time
# Однопоточная обработка файлов (для сравнения)
total_sum = 0
for f in files:
    total_sum += process_file(f[0], f[1], f[2])
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 4min 8s, sys: 13.4 s, total: 4min 21s
Wall time: 4min 21s


In [11]:
%%time 
# Многопроцессорная обработка файлов
total_sum = 0
with Pool(processes=10) as pool:
    for r in pool.starmap(process_file, files):
        total_sum += r
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 13.7 ms, sys: 88.5 ms, total: 102 ms
Wall time: 49.8 s


In [14]:
%%time
# Многопроцессорная обработка файлов через ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor, as_completed
total_sum = 0
with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    futures = [executor.submit(process_file, f[0], f[1], f[2]) for f in files]

    # Получение результатов задач
    for future in as_completed(futures):
        total_sum +=  future.result()
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
CPU times: user 4min 46s, sys: 6.77 s, total: 4min 53s
Wall time: 4min 43s


### Выводы по сравнению однопоточной работы, через multiprocessing.Pool и через concurrent.futures.ThreadPoolExecutor

Однопоточная работа чуть лучше, чем concurrent.futures.ThreadPoolExecutor. При работе в один поток хотя бы одно ядро процессора было загружено почти на 100%, а остальные 0-20%. А при работе через concurrent.futures.ThreadPoolExecutor все ядра были загружены ~20%.

При работе через multiprocessing.Pool все ядра грузятся на 90-100%.

Предположительно виноват GIL, который не дает эффективно работать concurrent.futures.ThreadPoolExecutor - они просто мешают друг другу.